In [1]:
# Before we start, load our environment
%matplotlib inline
import os
import re
import sys
import argparse
import csv
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

/usr/lib/python3/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Read the organisms metadata.

We have a file called `patric_metadata_20180526_isolation_host_env.tsv` that we want to parse. It has the columns `['genome_id', 'isolation_source', 'host_name', 'Environment]` and the main data that we want to classify against are host_name and Environment. Isolation_source is the original classification in PATRIC. Host_name is sparse - not everything has a host name.

In [3]:
def read_classifications(cf):
    """
    Read the classifications file. We expect that this has 4 tab-separated values.
    :param cf: the classifications file
    :return: two dicts of the hostname and environment. Recall, not everything has a hostname
    """
    hostname = {}
    environment = {}
    with open(cf, 'r') as f:
        for l in f:
            if l.startswith('genome_id'):
                continue
            p=l.strip().split("\t")
            if p[2]:
                hostname["PATRIC|{}".format(p[0])] = p[2]
            if p[3]:
                environment["PATRIC|{}".format(p[0])] = p[3]
    return hostname, environment

hostname, environment = read_classifications('patric_data/patric_metadata_isolation_host_env.tsv')                

# Read the focus output tsv file

We have a directory of focus output files that are compressed, and we parse those into a single .tsv file that has one genome per line and the columns are the metagenomes.

We parse that into a datastructure automatically. Note that telling pandas we will use the first row as header (header = 0) and the first column as the index (index_col = 0) are key to later pandas magic.

Note that our original file [metagenome_counts_20180625.tsv.gz](metagenome_counts_20180625.tsv.gz) is super huge - (28,705 by 18,920) and so we don't want to use that here. I wrote a [small script](create_dev_dataset.py) to pull out 100 metagenomes and all genomes that are not zero in those metagenomes. It chooses the 99 at random and you can also change the number of metagenomes selected. In my example, [dev_counts.tsv](dev_counts.tsv) the data set is (99 x 3,029) and so much, much faster!

In [23]:
def read_focus_output(fof):
    """
    :param fof: focus output file
    :return: a pandas data frame with the data
    """
    
    df = pd.read_csv(fof, sep="\t", header=0, index_col=0,)
    return df


# NOTE: The following file is (28,705 x 18,920 and so takes a long time to read!)
# focus = read_focus_output("metagenome_counts.tsv")
# this file is 3,029 genomes x 99 metagenomes
focus = read_focus_output('example_data/dev_counts_sel.tsv')
focus.head(10)

,SRR062179,ERR1045847,SRR5325943,SRR5813385,SRR2938409,ERR997363,SRR5325922,ERR996753,SRR1620020,ERR973049,...,ERR997017,ERR480834,SRR1571081,ERR1711763,SRR5326177,SRR5325957,ERR996752,ERR1045817,ERR1018308,ERR997287
PATRIC|42094.4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1453498.8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.127364,0.0
PATRIC|1579366.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.286779,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1946695.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|165186.16,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1262824.3,0.0,0.0,0.0,0.0,0.0,0.0,0.185172,0.000000,0.0,0.0,...,0.000000,0.039828,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1953362.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1961453.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.125815,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1986616.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1262685.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.508667,0.0,0.0,0.000000,0.0


In [5]:
focus['environment'] = pd.Series(environment)
focus['hostname'] = pd.Series(hostname)

In [24]:
focus.head(10)

,SRR062179,ERR1045847,SRR5325943,SRR5813385,SRR2938409,ERR997363,SRR5325922,ERR996753,SRR1620020,ERR973049,...,ERR997017,ERR480834,SRR1571081,ERR1711763,SRR5326177,SRR5325957,ERR996752,ERR1045817,ERR1018308,ERR997287
PATRIC|42094.4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1453498.8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.127364,0.0
PATRIC|1579366.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.286779,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1946695.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|165186.16,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1262824.3,0.0,0.0,0.0,0.0,0.0,0.0,0.185172,0.000000,0.0,0.0,...,0.000000,0.039828,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1953362.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1961453.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.125815,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1986616.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
PATRIC|1262685.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.508667,0.0,0.0,0.000000,0.0


In [7]:
# write this to a csv so I can get help!
focus.to_csv("/home/redwards/Desktop/metagenomes_genomes.tsv", sep="\t")

# NOTE: THIS IS WRONG!

We want to classify the metagenomes by the genomes, not vice versa! We're doing it wrong here. SEE BELOW

# Extract the features that we want to use in our random forest

Now that we have merged everything, we extract the column names of the features that we want to use in our random forest. This just creates an index of the species that we have. Note that if you use genus you will need to change the 492 to something else!

In [8]:
features = focus.columns[:-2]

# Create test and training sets.

For our classifier, we are going to use some part of the data to train the random forest classifier, and some part of the data to test it. We are going to make a new column that says whether it is testing or training, and populate it so that 75% of the data is training and 25% of the data is for testing. You can change those variables here. 

Note that in this example we are using the same data sets for training and testing. Before publication we should come up with sets of examplar metagenomes from each environment that we have manually curated and use those to train our classifiers.

In [9]:
focus['is_train'] = np.random.uniform(0, 1, len(focus)) <= .75
train, test = focus[focus['is_train']==True], focus[focus['is_train']==False]
print("Data: {}\nTraining: {}\nTesting: {}".format(focus.shape, train.shape, test.shape))

Data: (1942, 54)
Training: (1458, 54)
Testing: (484, 54)


# Create a factorized list of environments

The random forest requires the environments to just be a list of integers rather than labels, and so we use factorize to split out the labels and the indexes. This will be the input to our random forest classifier. Note that we are just using the training set here, not the whole data frame.

In [10]:
envfactors, labels = pd.factorize(train['environment'])
labels

Index(['animal', 'other water', 'human gut', 'soil', 'waste water', 'mud',
       'plant', 'sediment', 'human', 'human blood', 'human mouth', 'food',
       'sludge', 'human vagina', 'fresh water', 'hot spring',
       'chemical vessel', 'liquid environmental material', 'built environment',
       'human urogenital', 'marine coastal', 'marine', 'human nose', 'spring',
       'marine deep', 'biofilm', 'hypersaline lake',
       'solid environmental material', 'rock', 'human wound', 'oil', 'sand',
       'air', 'marine benthic', 'human lungs', 'mine', 'saltern',
       'human bone'],
      dtype='object')

# Start a random classifier

I leave most of this at the default (which is to use Gini as the measure of quality), and to bootstrap the trees.

In [11]:
clf = RandomForestClassifier(n_jobs=-1)

# Train the random forest

Now we train our random forest on the training data using just the features that we are looking for and with our factorized list of diagnoses

In [12]:
clf.fit(train[features], envfactors)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# Test our classifier using the test data set

This makes predictions for each of the test data sets. For each sample we get a number depending on which environment we predict it is from.

Then we convert those predictions to the appropriate labels. We now have an array of predictions the same length as our test data set.

In [13]:
clf.predict(test[features])
predictions=labels[clf.predict(test[features])]
predictions[0:10]

Index(['human gut', 'animal', 'human gut', 'other water', 'animal',
       'human gut', 'animal', 'human gut', 'waste water', 'human urogenital'],
      dtype='object')

And then we can compare those predictions to the test data set. As predicted we confuse 'human gut' with 'waste water'. Looks like we need a confusion matrix!

In [14]:
test.environment[0:10]

PATRIC|1946695.3      human gut
PATRIC|1262824.3      human gut
PATRIC|1262924.3      human gut
PATRIC|406547.6           plant
PATRIC|88431.5        human gut
PATRIC|1965562.3         animal
PATRIC|1801918.3    other water
PATRIC|853.91             human
PATRIC|1712665.3         animal
PATRIC|562.7691       human gut
Name: environment, dtype: object

## Why is this wrong?

In this random forest, we are predicting the source of the bacteria based on the environments that they come from ... i.e. we are around the wrong way.

We want to predict the source of the environments based on the bacteria that they contain.


# Recreate the data and rotate it.

Here, I just call the methods from above to get the data.

In [18]:
hostname, environment = read_classifications('patric_data/patric_metadata_isolation_host_env.tsv')                
focus = read_focus_output('example_data/dev_counts.tsv')
focus.head()

,ERR479276,SRR4022142,SRR061151,SRR2188733,SRR1612046,SRR5813477,SRR1748720,ERR1726969,SRR2151197,ERR517428,...,ERR517431,SRR1632481,SRR2188974,SRR2813243,ERR479473,ERR458965,SRR5574089,SRR3988600,SRR001040,SRR304674
PATRIC|42094.4,0.0,0.0,0.185454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.223634
PATRIC|1797517.3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013418
PATRIC|178900.7,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
PATRIC|1946695.3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
PATRIC|1079.6,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [19]:
ft = focus.T
ft.head()

,PATRIC|42094.4,PATRIC|1797517.3,PATRIC|178900.7,PATRIC|1946695.3,PATRIC|1079.6,PATRIC|1262824.3,PATRIC|1986779.3,PATRIC|1961453.3,PATRIC|93220.26,PATRIC|1618399.3,...,PATRIC|48408.3,PATRIC|1950755.3,PATRIC|1739515.3,PATRIC|1803481.3,PATRIC|575612.3,PATRIC|1231391.4,PATRIC|1262695.3,PATRIC|1950912.3,PATRIC|1163745.3,PATRIC|1483596.4
ERR479276,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR4022142,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR061151,0.185454,0.0,0.0,0.0,0.0,0.0,0.0,0.231997,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR2188733,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SRR1612046,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
